In [ ]:
from util.img_util import ImageDataLoader
from util.feature_A import crop
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
load_dotenv()

images_path = os.getenv("IMAGE_DATA_URL_LOCAL")
mask_path = os.getenv("MASK_DATA_URL_LOCAL")

In [ ]:
loader = ImageDataLoader(images_path, mask_path, hairless=False, augmentation=True)

working_img = "../images/PAT_233_354_828.png"
img_id, img_rgb, manual_mask, mask, noise_img, contrast_img, contrast_mask, extra_border_mask, assymetry, _border, color, assymetry_noise, _border_noise, color_noise, assymetry_contrast, _border_contrast, color_contrast, assymetry_extra_border, _border_extra_border, color_extra_border = loader.one_image(working_img)

In [ ]:
labels = ["Original", "Augmented extra border"]

original_text = f"A = {round(assymetry, 2)}, B = {round(_border, 2)}, C = {round(color, 5)}"
augmented_text = f"A = {round(assymetry_extra_border, 2)}, B = {round(_border_extra_border, 2)}, C = {round(color_extra_border, 5)}"

abc = [original_text, augmented_text]

# Create subplot with 3 columns, 1 row
fig, axes = plt.subplots(1, 2, figsize=(7, 5))

i = 0
for ax, img, label in zip(axes, [crop(mask), crop(extra_border_mask)], labels):
    ax.imshow(img, cmap="gray")
    ax.set_title(label, fontsize=14)
    ax.axis('off')
    ax.text(0.5, -0.1, abc[i], fontsize=12, ha='center', transform=ax.transAxes)
    i += 1

plt.tight_layout()
plt.show()

In [ ]:
labels = ["Original", "Augmented contrast"]

original_text = f"A = {round(assymetry, 2)}, B = {round(_border, 2)}, C = {round(color, 5)}"
augmented_text = f"A = {round(assymetry_contrast, 2)}, B = {round(_border_contrast, 2)}, C = {round(color_contrast, 5)}"

abc = [original_text, augmented_text]

# Create subplot with 3 columns, 1 row
fig, axes = plt.subplots(1, 2, figsize=(7, 5))

i = 0
for ax, img, label in zip(axes, [img_rgb, contrast_img], labels):
    ax.imshow(img)
    ax.set_title(label, fontsize=14)
    ax.axis('off')
    ax.text(0.5, -0.1, abc[i], fontsize=12, ha='center', transform=ax.transAxes)
    i += 1

plt.tight_layout()
plt.show()

In [ ]:
labels = ["Original", "Augmented noise"]

original_text = f"A = {round(assymetry, 2)}, B = {round(_border, 2)}, C = {round(color, 3)}"
augmented_text = f"A = {round(assymetry_noise, 2)}, B = {round(_border_noise, 2)}, C = {round(color_noise, 3)}"

abc = [original_text, augmented_text]

# Create subplot with 3 columns, 1 row
fig, axes = plt.subplots(1, 2, figsize=(7, 5))

i = 0
for ax, img, label in zip(axes, [img_rgb, noise_img], labels):
    ax.imshow(img)
    ax.set_title(label, fontsize=14)
    ax.axis('off')
    ax.text(0.5, -0.1, abc[i], fontsize=12, ha='center', transform=ax.transAxes)
    i += 1

plt.tight_layout()
plt.show()

In [ ]:
loader = ImageDataLoader(images_path, mask_path, hairless=True, augmentation=False)

working_img = "../sample_001.jpg"
img_id, img_rgb, tresh, img_rgb_hairless, manual_mask, mask, assymetry, _border, color = loader.one_image(working_img)

In [ ]:
loader = ImageDataLoader(images_path, mask_path, hairless=True, augmentation=False)

working_img = "../images/PAT_1402_1376_160.png"
img_id, img_rgb, tresh, img_rgb_hairless, manual_mask, mask, assymetry, _border, color = loader.one_image(working_img)

In [ ]:
print(assymetry, _border, color)
plt.imshow(mask, "gray")
plt.axis("off")

print(mask)

In [ ]:
labels = ["Original", "Binary", "Hair removed"]

# Create subplot with 3 columns, 1 row
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

for ax, img, label in zip(axes, [img_rgb, tresh, img_rgb_hairless], labels):
    ax.imshow(img, cmap="gray")
    ax.set_title(label)
    ax.axis('off')  # Hide axis ticks

plt.tight_layout()
plt.show()

In [ ]:
labels = ["Original", "Hair removed"]

# Create subplot with 3 columns, 1 row
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

for ax, img, label in zip(axes, [img_rgb, img_rgb_hairless, mask], labels):
    ax.imshow(img, cmap="gray")
    ax.set_title(label)
    ax.axis('off')  # Hide axis ticks

plt.tight_layout()
plt.show()

## Box plot to compare classifiers

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

metrics_csv = Path("..") / "data" / "result-OQ-smote.csv"
df = pd.read_csv(metrics_csv)

# gather F1 / AUC into long form
f1_cols  = [c for c in df.columns if c.startswith("F1_fold")]
auc_cols = [c for c in df.columns if c.startswith("AUC_fold")]

long_f1  = df.melt(id_vars="Model", value_vars=f1_cols,
                   var_name="Fold", value_name="Value")
long_f1["Metric"] = "F1"

long_auc = df.melt(id_vars="Model", value_vars=auc_cols,
                   var_name="Fold", value_name="Value")
long_auc["Metric"] = "AUC"

long_df = pd.concat([long_f1, long_auc])

# colors to use
box_color = "purple"
median_color = "black"

fig, axes = plt.subplots(2, 1, figsize=(10, 15), sharey=False)

for ax, metric in zip(axes, ["AUC", "F1"]):
    data = long_df[long_df["Metric"] == metric]
    bp = data.boxplot(
        column="Value",
        by="Model",
        ax=ax,
        patch_artist=True,               
        boxprops=dict(facecolor=box_color, color=box_color),
        medianprops=dict(color=median_color),
        whiskerprops=dict(color=box_color),
        capprops=dict(color=box_color),
        flierprops=dict(markeredgecolor=box_color)
    )
    ax.set_title(f"{metric} across folds")
    ax.set_xlabel("Model")
    ax.set_ylabel(metric)
    ax.tick_params(axis="x", rotation=90)

    if metric == "AUC":
        ax.set_ylim(0.3, 0.9)
    if metric == "F1":
        ax.set_ylim(-0.1, 0.5)

plt.suptitle("Box-plots of AUC and F1 per fold OQ SMOTE Model")
plt.tight_layout()
plt.show()

## Comparasion matrix for features

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


features_csv = Path("..") / "data" / "baseline-data-for-model.csv"
df = pd.read_csv(features_csv)

# Rename features if needed
df = df.rename(columns={
    'Z_feature_a': 'Asymmetry',
    'Z_feature_b': 'Border',
    'Z_feature_c': 'Color'
})

# Create PairGrid
g = sns.PairGrid(df, vars=['Asymmetry', 'Border', 'Color'], hue='Melanoma', palette={0: 'steelblue', 1: 'red'})

# Scatter plots in lower triangle
g.map_lower(sns.scatterplot, alpha=0.8, s=30)

# KDE plots on diagonal
g.map_diag(sns.kdeplot, common_norm=False, fill=True, alpha=0.4)

# upper triangle
g.map_upper(sns.scatterplot, alpha=0.8, s=30)

g.add_legend(title='Melanoma')
plt.suptitle("Asymmetry, Border, and Color by Melanoma Standarized", y=1.02)
plt.tight_layout()
plt.show()

## Example of inaccurate mask

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load the original and augmented images
original_img = mpimg.imread('pictureplot/image.png')
augmented_img = mpimg.imread('pictureplot/image copy.png')

# Create side-by-side plots
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Titles for each image
titles = ['Original', 'Mask']
images = [original_img, augmented_img]

for ax, title, image in zip(axes, titles, images):
    ax.imshow(image)
    ax.axis('off')
    ax.set_title(title, fontsize=14)

plt.tight_layout()
plt.show()